In [1]:
import sys, os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np
import h5py as hf
# %matplotlib notebook
%matplotlib tk
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
from foundation import sim as SIM
#from foundation.util import replicate, Cloner

np.set_printoptions(linewidth=120)

In [9]:
pts = torch.randn(10,2)
pts.shape

torch.Size([10, 2])

In [10]:
sphr = util.cart2sphr(pts)
sphr.shape

torch.Size([10, 2])

In [11]:
cart = util.sphr2cart(sphr)
cart.shape

torch.Size([10, 2])

In [12]:
nn.MSELoss()(cart, pts)

tensor(8.5598e-15)

In [13]:
pts

tensor([[-0.9066, -0.1331],
        [ 0.3564, -1.0869],
        [ 2.1459,  0.5159],
        [ 0.1939, -0.1277],
        [-0.8157, -0.2721],
        [ 0.3785, -1.6743],
        [ 0.9632, -0.7964],
        [ 0.5483,  0.3598],
        [-0.6143, -1.4157],
        [-0.8995, -1.1065]])

In [14]:
sphr

tensor([[ 0.9163, -2.9958],
        [ 1.1438, -1.2539],
        [ 2.2070,  0.2360],
        [ 0.2322, -0.5823],
        [ 0.8599, -2.8196],
        [ 1.7165, -1.3484],
        [ 1.2498, -0.6909],
        [ 0.6558,  0.5808],
        [ 1.5432, -1.9802],
        [ 1.4260, -2.2534]])

In [26]:
net = models.make_MLP(10,3,hidden_dims=[20,30,10], nonlin='softplus')
net

Sequential(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): Softplus(beta=1, threshold=20)
  (2): Linear(in_features=20, out_features=30, bias=True)
  (3): Softplus(beta=1, threshold=20)
  (4): Linear(in_features=30, out_features=10, bias=True)
  (5): Softplus(beta=1, threshold=20)
  (6): Linear(in_features=10, out_features=3, bias=True)
)

In [29]:
for layer in net[:-1]:
    if isinstance(layer, nn.Linear):
        print(layer)

Linear(in_features=10, out_features=20, bias=True)
Linear(in_features=20, out_features=30, bias=True)
Linear(in_features=30, out_features=10, bias=True)


In [3]:
sim.sample_action()

tensor([0.8320])

In [4]:
sim.sample_state()

tensor([0.6750, 0.1724, 0.2784, 0.7560])

In [5]:
u = sim.sample_action()
u = torch.ones(1)*0
u

tensor([0.])

In [19]:
B = 16
state0 = torch.tensor([0,np.pi,1,0])
state0 = sim.sample_state(B)

In [23]:
x = sim.reset(state0)

In [24]:
bb = int(np.sqrt(B))
if bb**2 == B:
    fig, axes = plt.subplots(bb,bb)
else:
    fig, ax = plt.subplots(1)

In [25]:
for i in range(100):
    u = sim.sample_action(B)
    x = sim.step(u, N=1)
    imgs = sim.render(200,100)
    if len(imgs.shape) == 4:
        for img, ax in zip(imgs, axes.flat):
            ax.cla()
            ax.imshow(img)
            ax.axis('off')
    else:
        ax.cla()
        ax.imshow(imgs)
        ax.axis('off')
    plt.pause(0.03)

In [21]:
plt.cla()
plt.imshow(sim.render(200,100,index=0 if B is not None else None))
plt.axis('off')
pass

In [22]:
xs = []
for i in range(100):
    u = sim.sample_action(10)
    x = sim.step(u, N=1)
    xs.append(x)
    plt.cla()
    plt.imshow(sim.render(200,100, index=0 if B is not None else None))
    plt.axis('off')
    plt.pause(0.03)
x

tensor([[ -0.4271,   3.2266,   1.4794,   5.3090],
        [ -0.3441,   2.8913,   0.9614,   0.1135],
        [  0.8356,   3.8679,  -1.0499,   0.5847],
        [  0.6757,   0.8301,  -1.4260,   2.9928],
        [  0.1056,   2.0255,  -1.7367,  -7.1185],
        [  0.9208,   3.9876,   2.4905,   3.4231],
        [ -0.7255,   2.4377,   1.5289,   1.6305],
        [  0.6668,   2.8852,   4.6571,   6.4283],
        [  1.0004,   2.2665,   0.1500,   5.9317],
        [  0.4574,   3.2469,  -2.4065, -10.3370]])

In [23]:
X = torch.cat(xs)
X.shape

torch.Size([1000, 4])

In [2]:
dataroot = '/is/ei/fleeb/workspace/data'
name = 'dsprites_ndarray_co1sh3sc6or40x32y32_64x64.hdf5'
name = '3dshapes.h5'
name = 'dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz'
# DS = hf.File(os.path.join(dataroot, name))
DS = np.load(os.path.join(dataroot, name))
len(DS)

4

In [5]:
list(DS.keys())

['metadata', 'imgs', 'latents_classes', 'latents_values']

In [6]:
DS['imgs'].shape

[autoreload of foundation.data.collectors failed: Traceback (most recent call last):
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_instances(old, new, obj, visited)
  File "/is/ei/fleeb/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 300, in update_instances
    for obj in (ob

(737280, 64, 64)

In [3]:
for k in list(DS.keys()):
    print(k)
    print(k, DS[k].shape)

metadata


ValueError: Object arrays cannot be loaded when allow_pickle=False

In [73]:
DS['latents']['values'][0]

array([1. , 1. , 0.5, 0. , 0. , 0. ])

In [ ]:
DS.close()

In [43]:
torch.eye(10)[torch.randperm(10)]

tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [22]:
B, D, M = 4, 2, 2

class Test(nn.Module):
    def __init__(self):
        super().__init__()
        self.A = nn.Linear(D,M)
        self.B = self.A

model = Test()
X, Y = torch.randn(B,D), torch.randn(B, M)

In [27]:
model.A.weight.data[0,0] += 1
model.A.weight, model.B.weight

(Parameter containing:
 tensor([[ 3.0000, -0.3344],
         [-0.1594, -0.0811]], requires_grad=True), Parameter containing:
 tensor([[ 3.0000, -0.3344],
         [-0.1594, -0.0811]], requires_grad=True))

In [26]:
model.load_state_dict(model.state_dict())

<All keys matched successfully>

In [16]:
p = nn.utils.parameters_to_vector(model.parameters())

In [14]:
loss = nn.MSELoss()(model(X), Y)
loss

tensor(0.3756, grad_fn=<MseLossBackward>)

In [15]:
model.zero_grad()
loss.backward()

In [19]:
for param in model.parameters():
    print(param.grad)

tensor([[ 0.0097, -0.0273],
        [ 0.1568, -0.0930]])
tensor([0.1551, 0.0066])


In [14]:
outputs = []
def keep(m, x, y):
    outputs.append(y)

In [6]:
model = torchvision.models.squeezenet1_0(pretrained=True, progress=True).cuda()
# model.features.register_forward_hook(keep)
print(model)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [9]:
x = torch.rand(4,3,128,128).cuda()
x.shape

torch.Size([4, 3, 128, 128])

In [17]:
y = model(x)
y.shape

torch.Size([4, 1000])

In [19]:
outputs[0].shape

torch.Size([4, 512, 7, 7])

In [19]:
info = torch.load('rl/temp_xy.pth.tar')
info.keys()

dict_keys(['y', 'm_old', 'x', 'm_new', 'l_new', 'l_old'])

In [20]:
X, Y = info['x'], info['y']
X.shape, Y.shape

(torch.Size([2050, 4]), torch.Size([2050, 1]))

In [21]:
mn, mo = info['m_new'], info['m_old']
print(list(mn.parameters()))
print(list(mo.parameters()))

[Parameter containing:
tensor([[ -6.0671, -15.6417,   1.6947,  -0.5189]]), Parameter containing:
tensor([5.8610])]
[Parameter containing:
tensor([[-0.4195,  0.3034,  0.2341,  0.3355]]), Parameter containing:
tensor([0.1772])]


In [22]:
info['l_old'], info['l_new']

(tensor(28.8512), tensor(5.7119))

In [3]:
N,M,D = 20, 5, 3
with torch.no_grad():
    A = torch.randn(N,M)
    f = nn.Linear(M,D)
    b = f(A)
    m = nn.Linear(M,D)

In [5]:
F.mse_loss(m(A), b)

tensor(1.1172, grad_fn=<MseLossBackward>)

In [6]:
util.solve(A,b,out=m)

Linear(in_features=5, out_features=3, bias=True)

In [7]:
F.mse_loss(m(A), b)

tensor(3.1220e-14, grad_fn=<MseLossBackward>)

In [25]:
shapes, settings = nets.plan_deconv(out_shape=(3,160,120), channels=[8, 16, 32, 64], kernels=4, strides=2)
shapes

[(8, 10, 7), (16, 20, 15), (32, 40, 30), (64, 80, 60), (3, 160, 120)]

In [26]:
ms = nn.ModuleList([nn.ConvTranspose2d(**s) for s in settings])
ms

ModuleList(
  (0): ConvTranspose2d(8, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), output_padding=(0, 1))
  (1): ConvTranspose2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (2): ConvTranspose2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
)

In [29]:
X = torch.randn(1, *shapes[0])

In [30]:
x = X
print(x.size())
for m in ms:
    x = m(x)
    print(x.size())

torch.Size([12, 8, 10, 7])
torch.Size([12, 16, 20, 15])
torch.Size([12, 32, 40, 30])
torch.Size([12, 64, 80, 60])
torch.Size([12, 3, 160, 120])


In [22]:
print(x.shape)
print(m)

torch.Size([12, 8, 10, 8])
ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))


In [45]:
shapes, settings = nets.plan_conv((3,160,120), channels=[8, 16, 32, 64], kernels=3, strides=1, factors=2)
shapes

[(3, 160, 120), (8, 80, 60), (16, 40, 30), (32, 20, 15), (64, 10, 8)]

In [46]:
ms = nn.ModuleList([nn.Conv2d(**s) for s in settings])
ms

ModuleList(
  (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [47]:
X = torch.randn(1, *shapes[0])

In [48]:
x = X
print(x.size())
for m in ms:
    x = m(x)
    print(x.size())

torch.Size([1, 3, 160, 120])
torch.Size([1, 8, 160, 120])
torch.Size([1, 16, 160, 120])
torch.Size([1, 32, 160, 120])
torch.Size([1, 64, 160, 120])


In [ ]:
ms = nn.ModuleList([nn.Conv2d(**s) for s in settings])

In [3]:
class OU_Process:
    def __init__(self, K=3, std=0.5, theta=0.5):
        self.K = K
        self.std = std
        self.theta = theta
        
        self.x = torch.randn(K) * self.std
        
    def __call__(self, t, state):
        return self.theta * self.x + torch.randn(self.K) * self.std
    
def fig2data ( fig ):
    """
    @brief Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
    @param fig a matplotlib figure
    @return a numpy 3D array of RGBA values
    """
    # draw the renderer
    fig.canvas.draw ( )
 
    # Get the RGBA buffer from the figure
    w,h = fig.canvas.get_width_height()
    buf = np.fromstring ( fig.canvas.tostring_argb(), dtype=np.uint8 )
    buf.shape = ( w, h,4 )
 
    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
    buf = np.roll ( buf, 3, axis = 2 )
    return buf

In [4]:
#env.alpha = 5.
#env.beta = 5.
#env.gamma = 0.1

In [5]:
env.reset()#(-torch.ones(2)*np.pi/2, torch.zeros(2))
#policy = OU_Process(K=env.N, std=0.5, theta=0.7)
policy = lambda t, x: torch.rand(env.spec.action_space.size)*2 - 1
plt.ioff()

In [6]:
rgb, lbl = env.render(label=True)
rgb.shape, lbl.shape

((200, 200, 3), (200, 200))

In [7]:
env.obs

tensor([-0.9174,  0.9359, -0.3980,  0.3524, -0.0811,  2.2122, -0.5983,
         0.3812,  0.0092, -0.0228])

In [8]:
plt.ion()
plt.figure()
plt.imshow(rgb, interpolation='none')

plt.figure()
plt.imshow(lbl, interpolation='none')

In [9]:
plt.ion()
figax = plt.subplots(figsize=(3,3))
fig, ax = figax
ax.set_xticks([])
ax.set_yticks([])

[]

In [10]:
for t in range(1000):
    
    action = policy(t, None) #+ 1
    env.step(action)
    
    ax.set_title('{:.3f} {:.3f}'.format(action[0].item(), action[1].item()))
    
    env.render('onscreen', figax=figax)
    plt.pause(0.002)
    ax.cla()
    #break

In [10]:
plt.show()

In [8]:
rgb = fig2data(fig)
rgb.shape

/home/fleeb/anaconda3/envs/cse/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


(409, 400, 4)

In [7]:
plt.figure()
#plt.axis('off')
plt.imshow(env.render('rgb'))
plt.show()

In [7]:
env.get_ctrl_freq()

60.0